<a href="https://colab.research.google.com/github/lilimelgar/mappingPersonNames/blob/main/src/MappingPersonNames_adapted_for_GoogleColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Script to map two lists of person names**

This notebook contains the steps for mapping two lists of person names (ListA and ListB) and gives as a result a list of possible candidates with scores.

This script is written by Liliana Melgar-Estrada for the SKILLNET PROJECT (https://skillnet.nl/)

Last update: June 17, 2022

# Data preparation (externally, before importing)

Please read the instructions for data preparation in the "read.me" file in the Github repository: https://github.com/lilimelgar/mappingPersonNames/blob/main/README.md

# Import libraries

In [1]:
pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 174 kB 4.8 MB/s 
     |████████████████████████████████| 2.2 MB 44.0 MB/s 


# Upload the "data" directory of the Github repository to your own Google Drive
The data directory is here: https://github.com/lilimelgar/mappingPersonNames/tree/main/data
You should upload it with the same folder structure.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import matplotlib
import pandas as pd
import numpy as np
import re
import os
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
# import jellyfish

from IPython.display import display
from IPython.display import clear_output

import csv

from IPython.display import display, HTML
# display(HTML("<style>.container { width:95% !important; }</style>"))
# pd.options.display.max_columns = 10
pd.options.display.max_rows = 1000
# pd.options.display.width = 1000

# to add timestamp to file names
import time

# for progress bar (https://datascientyst.com/progress-bars-pandas-python-tqdm/)
from tqdm import tqdm
from time import sleep

# Import files

## Set data directories

In [ ]:
# Test data is located in the repository folder indicated in the path here
# this is the local path to the raw data in your own computer to where you downloaded/cloned the repository
data_directory = os.path.abspath(os.path.join('..', 'data'))
data_raw_directory = os.path.join(data_directory, 'raw')
data_processed_directory = os.path.join(data_directory, 'processed')
data_temp_directory = os.path.join(data_directory, 'temp')

## Import ListA

For the test version, ListA contains unique names from the Catalogus Epistolarum Neerlandicarum (CEN) extracted from a slice of correspondents from van Leeuwenhoek and Swammerdam (internal note: cy08).

In [ ]:
# Import here the first file (ListA), this is the names you want to map the other list to.
# the list is imported as a pandas dataframe
list_a_path = os.path.join(data_raw_directory, 'ListA.csv')
dfA_t0 = pd.read_csv(list_a_path, sep = ",", index_col=False, engine='python')

In [ ]:
dfA_t0.info()

## Import list to map to (LIST B)

For the test version, ListB contains unique names from the Epistolarium (http://ckcc.huygens.knaw.nl/epistolarium/)  (internal note: cy13).

In [ ]:
# Import here the second file (ListB), these are the names you want to map (find a match) to the initial list.
# the list is imported as a pandas dataframe
list_b_path = os.path.join(data_raw_directory, 'ListB.csv')
dfB_t0 = pd.read_csv(list_b_path, sep = ",", index_col=False, engine='python')

In [ ]:
dfB_t0.info()

# Prepare ListA and ListB

In this step the data is prepared for the mappings (reassigning column names and changing data types in case they were not the right ones)

## Prepare ListA

In [ ]:
# assign column names
dfA_t0.columns = ['personIdA',
                   'nameStringA',
                   'dateBirthA', 
                   'dateDeathA', 
                   'dateFlA'
                   ]

In [ ]:
# make a copy of the dataframe and rename it
dfA = dfA_t0.reset_index(drop=True)

In [ ]:
# convert datatypes and fill in empty values
dfA_columns = dfA.columns
for column in dfA_columns:
    dataType = dfA.dtypes[column]
    if dataType == np.float64:
        dfA[column] = dfA[column].fillna(0.0)
        dfA[column] = dfA[column].astype(int)
    if dataType == object:
        dfA[column] = dfA[column].fillna('null')
        dfA[column] = dfA[column].astype(str)

In [ ]:
dfA.info()

In [ ]:
dfA.head(10)

## Prepare ListB

In [ ]:
# assign column names
dfB_t0.columns = [
                   'personIdB',
                   'nameStringB', 
                   'dateBirthB', 
                   'dateDeathB', 
                   'dateFlB',
                   ]

In [ ]:
# make a copy of the dataframe and rename it
dfB = dfB_t0.reset_index(drop=True).copy()

In [ ]:
# convert datatypes and fill in empty values
dfB_columns = dfB.columns
for column in dfB_columns:
    dataType = dfB.dtypes[column]
    if dataType == np.float64:
        dfB[column] = dfB[column].fillna(0.0)
        dfB[column] = dfB[column].astype(int)
    if dataType == object:
        dfB[column] = dfB[column].fillna('null')
        dfB[column] = dfB[column].astype(str)

In [ ]:
dfB.info()

In [ ]:
dfB.head(10)

## Store listA and listB for future reference

In [ ]:
# this inserts the timestamp in the file name
timestr = time.strftime("%Y%m%d-%H%M%S")

fileListA = (f"{data_temp_directory}/ListA_{timestr}.csv")
dfA.to_csv(fileListA)

fileListB = (f"{data_temp_directory}/ListB_{timestr}.csv")
dfB.to_csv(fileListB)

## Create a dataframe to store the mappings

# Run mapping script

Here below there is the mapping script that will compare the names in listB with the names in listA checkign if the name string matches and, if so, it applies some rules to determine if the respective dates of birth/death/fl. have a logical relation. If so, a mapping candidate is added to the dataframe C.

This script is also stored separately here: 

The counter shows:
|percentage done|items processed/total items \[time passed < time left, number of iterations per second\]

In [ ]:
mapped_candidates = compare_names(dfA, dfB)

# if you want to use different buffers, overwrite them
# mapped_candidates_buffer = compare_names(dfA, dfB, buf4 =8)

In [ ]:
mapped_candidates.info()

In [ ]:
mapped_candidates.head(5)

In [ ]:
# test = mapped_candidates[mapped_candidates.scoreCase.str.contains('L-')]

In [ ]:
# test

In [ ]:
mapped_candidates.scoreCase.value_counts()

# Prepare mapping output for analysis

#### Replace the .0 in person dates and convert to strings

In [ ]:

mapped_candidates['dateBirthA'] = mapped_candidates['dateBirthA'].astype(str).replace('\.0', '', regex=True)
mapped_candidates['dateDeathA'] = mapped_candidates['dateDeathA'].astype(str).replace('\.0', '', regex=True)
mapped_candidates['dateFlA'] = mapped_candidates['dateFlA'].astype(str).replace('\.0', '', regex=True)
mapped_candidates['match_dateBirthB'] = mapped_candidates['match_dateBirthB'].astype(str).replace('\.0', '', regex=True)
mapped_candidates['match_dateDeathB'] = mapped_candidates['match_dateDeathB'].astype(str).replace('\.0', '', regex=True)
mapped_candidates['match_dateFlB'] = mapped_candidates['match_dateFlB'].astype(str).replace('\.0', '', regex=True)


In [ ]:
mapped_candidates.info()

#### Create joined / unique names and fill the blanks

In [ ]:
mapped_candidates['JoinedInitial'] = mapped_candidates['nameStringA'] + '^' + mapped_candidates['dateBirthA'] + '^' + mapped_candidates['dateDeathA'] + '^' + mapped_candidates['dateFlA']
mapped_candidates['JoinedMapped'] = mapped_candidates['match_nameStringB'] + '^' + mapped_candidates['match_dateBirthB']  + '^' + mapped_candidates['match_dateDeathB'] + '^' + mapped_candidates['match_dateFlB']

# Fill in blanks
mapped_candidates['JoinedMapped'] = mapped_candidates['JoinedMapped'].fillna('notmapped')

In [ ]:
mapped_candidates.head(5)

In [ ]:
# mapped_candidates.info()

#### Run the second script to detect variation in the mapped forms

In [ ]:
# Convert these joined names to strings
mapped_candidates['JoinedInitial'] = mapped_candidates['JoinedInitial'].astype('string')
mapped_candidates['JoinedMapped'] = mapped_candidates['JoinedMapped'].astype('string')

In [ ]:
mapped_candidates.info()

In [ ]:
# create a score to determine how much the initial and the map form of the name (including dates) varies
for j in mapped_candidates.index:
    clear_output(wait=True)
    rowIndex = mapped_candidates.index[j]
    initialForm = mapped_candidates.iloc[j,13]
    mappedForm = mapped_candidates.iloc[j,14]
    matchScoreFinal = fuzz.ratio(initialForm, mappedForm)
    print("Current progress loop1:", np.round(j/len(mapped_candidates) *100, 2),"%")
    if 0 <= matchScoreFinal <=100:
        mapped_candidates.loc[rowIndex, 'ScoreMappedVersionsNotChangedis100'] = matchScoreFinal        

In [ ]:
# mapped_candidates

In [ ]:
mapped_candidates.columns

In [ ]:
# Reorder the columns in a way that is easier to evaluate mapping

dfD = mapped_candidates[['JoinedInitial',
        'JoinedMapped',
        'personIdA',
        'match_personIdB',
        'scoreCase',
        'scoreType',
        'scoreNameString',
        'ScoreMappedVersionsNotChangedis100']]


In [ ]:
dfD.info()

In [ ]:
dfD.scoreCase.value_counts()

# Download the mapping candidates file

In [ ]:
# This file will contain the mapping candidates, which is easier to evaluate externally, e.g., in OpenRefine

datasetA = 'ListA' #change list name if wanted
datasetB = 'ListB' #change list name if wanted
description = '' #add file description if wanted

#####bring back dfD
timestr = time.strftime("%Y%m%d-%H%M%S")
fileNameMappingCandidates = (f"{data_processed_directory}/MappingsPersonsCandidates_{datasetA}vs{datasetB}_{description}_{timestr}.csv")
dfD.to_csv(fileNameMappingCandidates)